<a href="https://colab.research.google.com/github/nassma2019/PracticalSessions/blob/master/vision/baseline_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf

In [2]:
#@title Loading the cifar10 dataset and minor pre-processing.
(train_inputs, train_labels), (test_inputs, test_labels) = tf.keras.datasets.cifar10.load_data()
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels)
test_labels_one_hot = tf.keras.utils.to_categorical(test_labels)

# Normalizing inputs into the [-1, 1.] range.
train_inputs = 2 * (train_inputs / 255.) - 1
test_inputs = 2 * (test_inputs / 255.) - 1

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
#@title Model architecture.
FILTERS = [64, 64, 128, 128, 128, 256, 256, 256, 512, 512, 512] #@param
STRIDES = [1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1] #@param
NUM_CLASSES = 10 #@param

assert len(FILTERS) == len(STRIDES)
NUM_LAYERS = len(FILTERS)

In [6]:
#@title Model definition.
tf.reset_default_graph()
x = tf.keras.layers.Input(shape=(32, 32, 3))

print("Filters: %s" % FILTERS)
print("Strides: %s" % STRIDES)
print("")
print("Input shape = %s" % x.shape)
print("")

y = x
for i in range(NUM_LAYERS):
  print("Layer #%d:" % i)
  y = tf.keras.layers.Conv2D(filters=FILTERS[i], strides=STRIDES[i],
                             kernel_size=(3, 3), padding='SAME')(y)
  y = tf.keras.layers.BatchNormalization()(y)
  y = tf.keras.layers.ReLU()(y)
  print("  - Output shape = %s" % y.shape)

print("")
y = tf.keras.layers.Lambda(lambda t : tf.reduce_mean(t, reduction_indices=[1, 2]),
                          name='reduce_mean')(y)
print("Shape after `reduce_mean`: %s" % y.shape)
y = tf.keras.layers.Dense(NUM_CLASSES)(y)
y = tf.keras.layers.Softmax()(y)
print("Final output shape: %s" % y.shape)
model = tf.keras.Model(inputs=x, outputs=y)

Filters: [64, 64, 128, 128, 128, 256, 256, 256, 512, 512, 512]
Strides: [1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1]

Input shape = (?, 32, 32, 3)

Layer #0:
  - Output shape = (?, 32, 32, 64)
Layer #1:
  - Output shape = (?, 32, 32, 64)
Layer #2:
  - Output shape = (?, 16, 16, 128)
Layer #3:
  - Output shape = (?, 16, 16, 128)
Layer #4:
  - Output shape = (?, 16, 16, 128)
Layer #5:
  - Output shape = (?, 8, 8, 256)
Layer #6:
  - Output shape = (?, 8, 8, 256)
Layer #7:
  - Output shape = (?, 8, 8, 256)
Layer #8:
  - Output shape = (?, 4, 4, 512)
Layer #9:
  - Output shape = (?, 4, 4, 512)
Layer #10:
  - Output shape = (?, 4, 4, 512)

Shape after `reduce_mean`: (?, 512)
Final output shape: (?, 10)


In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
re_lu (ReLU)                 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32, 32, 64)        0     

In [0]:
#@title Train the model.
# Note: does not converge as quickly as the original model.
# (no data augmentation / different optimizer)
optimizer = tf.keras.optimizers.Adagrad(lr=0.1, decay=1e-6)
model.compile(optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_inputs, train_labels_one_hot, batch_size=32, shuffle=True)